# EX13 Movie Recommendation  
  
## Objective  
  
1. CSR matrix가 정상적으로 만들어졌다.  
사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.  
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.  
사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.  
3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.  
MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도를 측정하고 의미를 분석해보았다.  
  
별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.  
별점을 시청횟수로 해석해서 생각하겠습니다.  
또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.  

In [42]:
import pandas as pd

import numpy as np
import scipy
import implicit

### 1) 데이터 준비와 전처리

In [43]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [44]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [45]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [46]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [47]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


전처리가 완료되었습니다  
MF model을 구상한 후, 내가 좋아할 만한 영화를 추천해보는 것이 고ㅏ제

### 2) 분석하기  
#### ratings에 있는 유니크한 영화 개수

In [48]:
# 13-3의 nunique를 참고!
# 유저 수
ratings['movie_id'].nunique()

3628

#### ratings에 있는 유니크한 사용자 수

In [49]:
# 13-3의 nunique를 참고!
ratings['user_id'].nunique()

6039

이후 분석 과정에서 두 데이터프레임의 병합이 필요해보입니다  
합쳐준 후, 필요하지 않은 timestamp 컬럼은 삭제합니다

In [50]:
data = pd.merge(ratings, movies)
data = data.drop('timestamp', axis = 1)

In [51]:
data

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836473,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,Slaughterhouse (1987),Horror
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary


#### 가장 인기 있는 영화 30개(인기순)  

In [52]:
# 각 영화의 'rating'를 합친걸 가져와야겠습니다
# 복잡하지만 groupby, sort_values 등으로 해결할 수 있을듯
# 13-3의 #인기많은 아티스트 부분 참고

pop_count = data.groupby('title')['counts'].count()
pop_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

인기많은 숫자는 일단 데이터에 붙여둘게요!

In [53]:
data = pd.merge(data, pd.DataFrame(pop_count), how='left', left_on='title', right_on='title')


In [54]:
data = data.rename(columns={'counts_x':'counts', 'counts_y':'total_counts'})
data.head(5)

,user_id,movie_id,counts,title,genre,total_counts
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama,1680
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama,1680
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama,1680
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama,1680
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama,1680


In [55]:
pop_count = data.groupby('title')['counts'].count()
pop_count.sort_values(ascending=False).head(10)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Name: counts, dtype: int64

데이터프레임에 모든 정보와 컬럼명이 잘 반영이 된거 같습니다  
  
### 3) 내가 선호하는 영화 5가지 추가하기  
  
내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍니다  
저는 rating과 title을 병합한 
추가되는 데이터를 잘 반영하기 위해 내가 뽑을 영화 정보도 같이 뽑아 데이터프레임에 전부 반영시키겠습니다

In [56]:
# 13-3의 'zimin이 등장하는' 부분에 답이 있습니다
my_favorite_title = ['Star Wars: Episode V - The Empire Strikes Back (1980)', 'Matrix, The (1999)','Pulp Fiction (1994)' ,'Braveheart (1995)' ,'Shakespeare in Love (1998)']

my_movies_info = pd.DataFrame()
for s in my_favorite_title:
    my_movies_info = pd.concat([my_movies_info , data[data['title'] == s]])

#내 정보가 들어갈 unique한 컬럼들 제거해주고~ 중복 드롭하면 예쁘게 필요한 정보들만 나올듯
my_movies_info = my_movies_info.drop('user_id', axis = 1)
my_movies_info = my_movies_info.drop('counts', axis = 1)

my_movies_info = my_movies_info.drop_duplicates()
my_movies_info

,movie_id,title,genre,total_counts
111764,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War,2885
122219,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,2434
203449,296,Pulp Fiction (1994),Crime|Drama,2030
83903,110,Braveheart (1995),Action|Drama|War,2314
125453,2396,Shakespeare in Love (1998),Comedy|Romance,2213


In [57]:
my_favorite_id = my_movies_info['movie_id'].values
my_favorite_genre = my_movies_info['genre'].values
my_favorite_total_counts = my_movies_info['total_counts'].values.copy()

print(my_favorite_total_counts)

#제 취향도 반영될테니 1 더해줄게요
for i in range(len(my_favorite_total_counts)):
    my_favorite_total_counts[i] += 1
    
print(my_favorite_total_counts)


[2885 2434 2030 2314 2213]
[2886 2435 2031 2315 2214]


In [58]:
# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.

my_playlist = pd.DataFrame({'user_id': ['kspark']*5, 'title': my_favorite_title, 'counts':[5]*5, 'movie_id':my_favorite_id, 
                            'genre':my_favorite_genre, 'total_counts':my_favorite_total_counts})

if not data.isin({'user_id':['kspark']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,title,genre,total_counts
836473,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western,1
836474,5854,3026,4,Slaughterhouse (1987),Horror,1
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance,1
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary,1
836477,5948,1360,5,Identification of a Woman (Identificazione di ...,Drama,1
0,kspark,1196,5,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War,2886
1,kspark,2571,5,"Matrix, The (1999)",Action|Sci-Fi|Thriller,2435
2,kspark,296,5,Pulp Fiction (1994),Crime|Drama,2031
3,kspark,110,5,Braveheart (1995),Action|Drama|War,2315
4,kspark,2396,5,Shakespeare in Love (1998),Comedy|Romance,2214


### 4) CSR matrix 직접 만들어 보기  
희소행렬을 만들기 위해 컬럼값을 indexing하는 전처리가 필요합니다  
지금까지 했던 작업에서는 간단히 저의 id만 바꿔주면 될 것 같습니다만  
노드에서 소개된 함수를 이용하여 전체를 바꾸는 방법을 진행해보겠습니다  

In [67]:
# 유저, 무비 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [64]:
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing OK!!')
    data['movie_id'] =  temp_movie_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,counts,title,genre,total_counts
0,0,0,5,One Flew Over the Cuckoo's Nest (1975),Drama,1680
1,1,0,5,One Flew Over the Cuckoo's Nest (1975),Drama,1680
2,2,0,4,One Flew Over the Cuckoo's Nest (1975),Drama,1680
3,3,0,4,One Flew Over the Cuckoo's Nest (1975),Drama,1680
4,4,0,5,One Flew Over the Cuckoo's Nest (1975),Drama,1680
...,...,...,...,...,...,...
0,6039,117,5,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War,2886
1,6039,124,5,"Matrix, The (1999)",Action|Sci-Fi|Thriller,2435
2,6039,222,5,Pulp Fiction (1994),Crime|Drama,2031
3,6039,87,5,Braveheart (1995),Action|Drama|War,2315


id가 0번부터 재할당 되었습니다  
제일 마지막이 저의 정보입니다  
앞서 확인한 유니크한 유저 6039명의 id가 0부터 다시 할당된 후, 저의 아이디가 할당되었습니다  

csr 매트릭스를 생성해줍니다

In [65]:
# 13-6 참고하기

from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

총 836483의 요소를 저장하는,  6040의 유니크한 유저수와 3628의 영화수를 반영하는 csr matrix가 생성되었습니다  

### 5) als_model 구성 및 훈련하기  
  
als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다  
이거 시간이 엄청 걸리는 부분이에요  

In [68]:
#모델 구축
from implicit.als import AlternatingLeastSquares

import os
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [69]:
help(AlternatingLeastSquares)

Help on function AlternatingLeastSquares in module implicit.als:

AlternatingLeastSquares(factors=100, regularization=0.01, dtype=<class 'numpy.float32'>, use_native=True, use_cg=True, use_gpu=True, iterations=15, calculate_training_loss=False, num_threads=0, random_state=None)
    Alternating Least Squares
    
    A Recommendation Model based off the algorithms described in the paper 'Collaborative
    Filtering for Implicit Feedback Datasets' with performance optimizations described in
    'Applications of the Conjugate Gradient Method for Implicit Feedback Collaborative
    Filtering.'
    
    This factory function switches between the cpu and gpu implementations found in
    implicit.cpu.als.AlternatingLeastSquares and implicit.gpu.als.AlternatingLeastSquares
    depending on the use_gpu flag.
    
    Parameters
    ----------
    factors : int, optional
        The number of latent factors to compute
    regularization : float, optional
        The regularization factor to use


In [70]:
#노드에 기본값으로 주어진 숫자들은 모델이 가진 초기화 값이었습니다

n_factors = 100              #유저와 아이템의 벡터를 몇 차원으로 할 것인지
n_regularization  = 0.01     #과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
n_iterations = 15            #epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

#새로 쓸 parameters

r_state = 2023

#저는 이터레이션을 조금 더 돌릴께요

n_iterations = 30


In [71]:
#모델 생성하고
als_model = AlternatingLeastSquares(factors=n_factors, regularization=n_regularization, calculate_training_loss = True, 
                                    use_gpu=False, iterations=n_iterations, dtype=np.float32, random_state = r_state)

In [72]:
#핏 = 트레인
als_model.fit(csr_data.T)
#item행 x user열 로 입력을 받기 때문에, 반대의 경우라면 transpose 해주세요


  0%|          | 0/30 [00:00<?, ?it/s]

### 6) 내가 선호하는 5가지 영화 중 하나와, 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해보기  
여러개 보겠습니다  
#### 내가 선택한 영화

In [75]:
#13-7의 queen이 나오는 부분, 벡터 내적

#내 아이디 6039..

test_user = user_to_idx[6039]
user_vector = als_model.user_factors[test_user]

for s in my_favorite_title:
    test_movie = movie_to_idx[s]
    movie_vector = als_model.item_factors[test_movie]
    res = np.dot(user_vector, movie_vector)
    print(s, " 를 선호하는 정도는 : ", res)

Star Wars: Episode V - The Empire Strikes Back (1980)  를 선호하는 정도는 :  0.56946445
Matrix, The (1999)  를 선호하는 정도는 :  0.5292536
Pulp Fiction (1994)  를 선호하는 정도는 :  0.37773126
Braveheart (1995)  를 선호하는 정도는 :  0.57046103
Shakespeare in Love (1998)  를 선호하는 정도는 :  0.49921957


좋아하는 영화라면 당연히 1에 가깝게 나와야 할까요?  
현재로서는 결과값이 의미있는지, 잘 모르겠습니다만  
  
#### 내가 선택하지 않았던 영화
안좋아하는 영화들 (좋아는 하지만 다른것보다는 덜 좋아하는 ㅎㅎ;) 것들과 비교해보겠습니다 (무서운거 드라마 안좋아함..)  
숫자가 조금 적게 나올까요?

In [76]:
my_soso_title = ['Fargo (1996)','Shawshank Redemption, The (1994)','Forrest Gump (1994)']

for s in my_soso_title:
    test_movie = movie_to_idx[s]
    movie_vector = als_model.item_factors[test_movie]
    res = np.dot(user_vector, movie_vector)
    print(s, " 를 선호하는 정도는 : ", res)

Fargo (1996)  를 선호하는 정도는 :  0.33615544
Shawshank Redemption, The (1994)  를 선호하는 정도는 :  0.3961378
Forrest Gump (1994)  를 선호하는 정도는 :  0.18922089


조금은 작은 숫자가 나왔습니다  
의미가 있네요!

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다

In [78]:
#13-8의 similar_items 
#artist_to_idx 를 뒤집어, index로부터 movies 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

In [82]:
#제가 좋아하는 스타워즈5와 비슷한 영화는요
similar_movie = als_model.similar_items(movie_to_idx[my_favorite_title[0]], N=15)
similar_movie

[(117, 1.0),
 (64, 0.8933744),
 (44, 0.8861404),
 (120, 0.6532162),
 (200, 0.43792716),
 (22, 0.39266807),
 (172, 0.39165938),
 (60, 0.38660994),
 (651, 0.38639948),
 (26, 0.35837302),
 (5, 0.35684812),
 (124, 0.34701356),
 (193, 0.3345868),
 (680, 0.3143431),
 (550, 0.30573642)]

1도 나오네요?  
영화명으로 변경해봅니다

In [83]:
[idx_to_movie[i[0]] for i in similar_movie]

['Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Raiders of the Lost Ark (1981)',
 'Terminator, The (1984)',
 'Back to the Future (1985)',
 'Indiana Jones and the Last Crusade (1989)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Aliens (1986)',
 'E.T. the Extra-Terrestrial (1982)',
 'Princess Bride, The (1987)',
 'Matrix, The (1999)',
 'Alien (1979)',
 'Blade Runner (1982)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)']

Instruction에 '추천' 이라고 명시되어 있습니다만, 사실 이 리스트가 내포하는 것은 '비슷함' 일 것입니다  
리스트에는 스페이스 오페라, SF 장르등이 선별되었습니다    
특히 스타워즈 시리즈가 상단부에 있는 것이 너무 재밌습니다  

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다

In [84]:
#13-8의 recommend 메소드 

movie_recommended = als_model.recommend(test_user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(48, 0.6058494),
 (64, 0.5749327),
 (44, 0.54287934),
 (92, 0.52171767),
 (99, 0.5037916),
 (23, 0.4202258),
 (157, 0.39613777),
 (121, 0.3955263),
 (5, 0.36124673),
 (120, 0.35242265),
 (51, 0.33615544),
 (224, 0.3030842),
 (107, 0.29070723),
 (110, 0.2878543),
 (141, 0.2834292),
 (269, 0.26543963),
 (200, 0.26224568),
 (233, 0.26014364),
 (22, 0.2564019),
 (175, 0.25472856)]

In [85]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Saving Private Ryan (1998)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Terminator 2: Judgment Day (1991)',
 'American Beauty (1999)',
 "Schindler's List (1993)",
 'Shawshank Redemption, The (1994)',
 'Silence of the Lambs, The (1991)',
 'Princess Bride, The (1987)',
 'Raiders of the Lost Ark (1981)',
 'Fargo (1996)',
 'L.A. Confidential (1997)',
 'Jurassic Park (1993)',
 'Groundhog Day (1993)',
 'Fugitive, The (1993)',
 'GoodFellas (1990)',
 'Terminator, The (1984)',
 'Usual Suspects, The (1995)',
 'Back to the Future (1985)',
 'Men in Black (1997)']

재밌는 영화들이 추천되었습니다..만, 제가 덜 좋아한다고 써놨던 Fargo와 쇼생크탈출도 리스트에 있었습니다  
하지만 모델 학습을 더 한다고 뚜렷한 개선사항이 생길지는 잘 모르겠습니다..

## 회고  
즐거운 학습이었습니다  
  
### 인상깊었던 점   
ALS model 학습 시간이 굉장히 빨랐습니다  
80만개의 단일 (x, y) 좌표형 데이터를 학습시키는데 다양한 regrssion 알고리즘들이나 nn이 가져가는 cost는 얼마나 될까요?  
엄청나게 느리지 않을수도 있겠지만, 1 epoch에 1초 내외를 소비하는 ALS model에 비해서는 많이 느릴 것 같습니다  
  
또한 영화의 세세한 feature로 사용할 수 있는 특징들, 예를 들면 장르라던가, 러닝타임, 캐스팅 등의 요소를 고려하지 않아도,  
이미 내가 보았다는 사실을 기반으로 rough한 추천을 할 수 있었던 점이 인상적이었습니다 
  
### 아쉬운 점   
  
#### 모델   

학습을 더 한다고 뚜렷한 개선사항이 생길 지 모르겠는 이유는, 이미 모델이 overfitting을 겪은 상황으로 판단하기 떄문입니다  
모델의 training loss는 20epoch 대에서 0.3 이하로 떨어졌다가, 0.37까지 다시 오른 상태에서 트레인을 마쳤습니다  
따라서 매우 쉽게 overfitting을 겪는다는 판단이 생겼습니다  
  
#### 라이브러리   
  
https://implicit.readthedocs.io/en/latest/als.html  
해당 라이브러리의 페이지입니다  
parameter 조정을 할 수 있는 것들이 다양하지 않았습니다  

라이브러리의 다른 아쉬웠던 점은 loss등 training 과정에서 일어나는 일들을 남겨서 추이를 확인할 수 없었다는 점 (실시간 디스플레이만 가능)  
과 grid search등의 hyperparameter tuning 이 지원되지 않는다는 점이었습니다  
참고 https://stackoverflow.com/questions/60211437/gridsearchcv-with-als-algorithm


### 기대되는 점  
#### 다른 라이브러리  
  
같은 ALS model을 구현한 다음 라이브러리에는 더 많은 옵션들이 있는 것 같아서,  
이후 ALS를 사용하게 될 상황에는 다음 라이브러리를 사용하게 될 것 같습니다  
https://spark.apache.org/docs/latest/ml-collaborative-filtering.html  
   
#### 추천결과   
마지막 8) 내가 가장 좋아할 만한 영화들을 추천받아보기 항목에서,  
결과가 좋지 않았던 중요한 요건중에 하나는 제 취향인 영화가 다섯개밖에 반영이 안되었기 때문이라고 생각합니다  
더 많은 영화를 반영할수록, 더 좋은 결과를 얻을 수 있을거라고 확신하고 있습니다  
그리고 이것이, 일반적으로 이야기 할 수 있는 추천시스템의 특성인듯 합니다  
